**Building a Neural Network that can classify whether the person is feeling Happy/Sad!!!**

In [0]:
#Importing the required packages
import tensorflow as tf #Tensorflow package for Deep Learning
import os #For Directory level operations
import zipfile #For Extracting the zipfiles

In [7]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

--2020-06-13 15:09:40--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 2607:f8b0:400c:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-06-13 15:09:40 (216 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [0]:
#Extracting zip file which was downloaded in previous cell
zip_location = zipfile.ZipFile('/tmp/happy-or-sad.zip', 'r')
zip_location.extractall("/tmp/Happy-Sad")
zip_location.close()

In [0]:
#Creating a Callback class to stop training when our required Accuracy reaches
class Callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.95):
      print('\n95% Accuracy reached!!!\nCancelling Training!')
      self.model.stop_training = True

callback = Callback()

In [0]:
#Starting building model with layers
model = tf.keras.models.Sequential([
                                  tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape = (150, 150, 3)),
                                  tf.keras.layers.MaxPooling2D(2, 2),
                                  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                                  tf.keras.layers.MaxPooling2D(2, 2),
                                  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                                  tf.keras.layers.MaxPooling2D(2, 2),
                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(512, activation='relu'),
                                  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
#First, import the RMSProp optimizer(As we use this for this project)
from tensorflow.keras.optimizers import RMSprop
#Compiling the model with all the layers set up
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics = ['accuracy'])

In [13]:
#Using Keras API(ImageDataGenerator) to resize all the images while training
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(
    "/tmp/Happy-Sad",
    target_size = (150, 150),
    batch_size = 10,
    class_mode = 'binary'
)

Found 80 images belonging to 2 classes.


In [15]:
#Now, Fitting the model
history = model.fit(train_generator,
                    steps_per_epoch = 8,
                    epochs = 15,
                    verbose = 1,
                    callbacks = [callback])

Epoch 1/15
8/8 [==============================] - ETA: 0s - loss: 0.0171 - accuracy: 1.0000
95% Accuracy reached!!!
Cancelling Training!
8/8 [==============================] - 1s 182ms/step - loss: 0.0171 - accuracy: 1.0000
